# Agenda

- Power Meter
    - 何謂power meter
    - 來看看Power Meter的點表
    - Live Demo
    
- MQTT - 即時性應用
    - Mosquitto
    - paho
    
- thingspeak

----


## 在進入主題之前
----
### modbus格式介紹 

[參考](http://gridconnect.com/blog/tag/modbus-rtu/)

- Modbus/RTU: [start time] [Address 8bits + Function 8bits + Data Nx8bits + CRC 16bits] [End time]

- Modbus/TCP: [header 6byte + Address 8bits + Function 8bits + Data Nx8bits]

- Modbus/ASCII: 現在比較少人用，跳過不講

![格式](image/MODBUS-Frame.png)
        


### modbus常被採用的原因
        
- 公開發表並且無版稅要求

- 相對容易的工業網路部署

- 協定格式簡單，極省資源


----

## Modbus套件

----



### 網路上較多人提到的三個套件

[performance比較](https://stackoverflow.com/questions/17081442/python-modbus-library)

- modbus-tk: Modbus/RTU, Modbus/TCP

- pymodbus: 據說實作最完整，但使用資源相對的多，相依套件也多

- MinimalModbus: Modbus/RTU, Modbus/ASCII



### Modbus-tk

個人推薦使用

- 安裝方式
    - pip install serial
    - pip install modbus_tk

- 操作
    - [Python與PLC共舞](https://github.com/maloyang/PLC-Python)
     

----

## Power Meter

電力監控常見的元素

----



### Power Meter的點表

![Power Meter的點表](image/power_meter.gif)



### Power Meter的浮點數表示方式

![浮點數](image/power_meter_float.gif)

----
![float](image/mb_float.png)
- [Live Demo](Modbus.ipynb)


In [ ]:
!pip install modbus_tk

In [4]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu # RS-485
import modbus_tk.modbus_tcp as modbus_tcp # 網路線 v
import time
from struct import *

In [3]:
# Modbus/TCP Client Start
# master (client) vs slave (server)

mbTimeout = 1
mb_ip = '209.97.161.199'
mb_port = 10502 #502

master = modbus_tcp.TcpMaster(mb_ip, mb_port)
master.set_timeout(mbTimeout)

mbId = 1
addr = 0
try:
    # FC3
    rr = master.execute(mbId, cst.HOLDING_REGISTERS, addr, 3)
    print('read value:', rr)

except Exception as e:
    print("modbus test Error: " + str(e))

master._do_close()


read value: (110, 112, 109)


True

In [7]:
# if float

# Modbus/TCP Client Start

mbTimeout = 1
mb_ip = '209.97.161.199'
mb_port = 10502

master = modbus_tcp.TcpMaster(mb_ip, mb_port)
master.set_timeout(mbTimeout)

mbId = 1
addr = 20
try:
    # FC3
    rr = master.execute(mbId, cst.HOLDING_REGISTERS, addr, 2)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        v_a_hi = rr[1]
        v_a_lo = rr[0]
        v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))[0]
        print('v_a=', v_a)
        print(round(v_a, 2))
        #struct.unpack(">f",'\x42\xd8\x6b\x8d')
    except Exception as e:
        print(e)
    
except Exception as e:
    print("modbus test Error: " + str(e))

master._do_close()


read value: (13107, 16387)
v_a= 2.049999952316284
2.05


True

In [15]:
# TODO: 如果address 20開始有三個float表示電流值，要怎麼處理?
# if float

# Modbus/TCP Client Start

mbTimeout = 1
mb_ip = '209.97.161.199'
mb_port = 10502

master = modbus_tcp.TcpMaster(mb_ip, mb_port)
master.set_timeout(mbTimeout)

mbId = 1
addr = 20
try:
    # FC3
    rr = master.execute(mbId, cst.HOLDING_REGISTERS, addr, 6)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        hi = rr[1]
        lo = rr[0]
        value = unpack('>f', pack('>HH', hi, lo))[0]
        print('value=', round(value, 2))

        hi = rr[3]
        lo = rr[2]
        value = unpack('>f', pack('>HH', hi, lo))[0]
        print('value=', round(value, 2))

        hi = rr[5]
        lo = rr[4]
        value = unpack('>f', pack('>HH', hi, lo))[0]
        print('value=', round(value, 2))
        
    except Exception as e:
        print(e)
    
except Exception as e:
    print("modbus test Error: " + str(e))

master._do_close()


read value: (13107, 16387, 39322, 16393, 28836, 16381)
type tuple doesn't define __round__ method


True

In [ ]:
[ 12098,65,0,0,0]
B/A

In [9]:
12098//256

47

In [10]:
12098%256

66

In [11]:
chr(47)

'/'

In [12]:
chr(66)

'B'

In [13]:
chr(65)

'A'

In [ ]:
0xA/B

----

## MQTT - 即時性應用

已經可以採集資料了，來談談怎麼上傳雲端吧

----


### Mosquitto

一個broker套件，當然也可以做為client使用

- 以NB X260來說，4000多個連結沒有問題

![Mosquitto](image/Eclipse-Mosquitto-logo.png)


### paho

便利的MQTT client端套件

- [link](https://pypi.org/project/paho-mqtt/)
- install: `pip install paho-mqtt`

![paho](image/mqtt-paho-featured-image.jpg)


In [16]:
import paho.mqtt.client as mqtt
import time, json, datetime, sys, os

#==================
#== MQTT Functions

# 當地端程式連線伺服器得到回應時，要做的動作
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    # 將訂閱主題寫在on_connet中
    # 如果我們失去連線或重新連線時 
    # 地端程式將會重新訂閱
    client.subscribe('KH20221202_IoT_Data_Science/#')
    

# 當接收到從伺服器發送的訊息時要進行的動作
def on_message(client, userdata, msg):
    # 轉換編碼utf-8才看得懂中文
    try:
        topic = msg.topic
        value = msg.payload.decode('utf-8')
        dtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print('(topic, value)=(%s, %s) @%s' %(topic, value, dtime))
    except Exception as e:
        print('read data exception: ', str(e))

def run_mqtt():
    try:
        # 連線設定
        # 初始化地端程式
        client = mqtt.Client()

        # 設定連線的動作
        client.on_connect = on_connect

        # 設定接收訊息的動作
        client.on_message = on_message

        # 設定登入帳號密碼
        #client.username_pw_set("try","xxxx")

        # 設定連線資訊(IP, Port, 連線時間)
        client.connect("broker.hivemq.com", 1883, 60)

        # 開始連線，執行設定的動作和處理重新連線問題
        # 也可以手動使用其他loop函式來進行連接
        #client.loop_forever()
        client.loop_start()    #start the loop

        time.sleep(30)

        client.disconnect() # disconnect gracefully
        client.loop_stop() # stops network loop
        
    except Exception as e:
        print('run mqtt error: ', str(e))

run_mqtt()


Connected with result code 0
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a1, 0.405) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm1/total_e, 5587.403) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v2, 233.302) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v3, 238.955) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a1, 0.384) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a3, 1.526) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm2/total_e, 18705.350) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm3/a1, 1.599) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm3/a2, 0.171) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm3/v1, 136.152) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/farm3/v2, 136.586) @2022-12-24 11:54:27
(topic, value)=(KH20221202_IoT_Data_Science/

In [23]:
my_dict = {'farm1':{}}
my_dict['farm1']['v1'] = 220
print(my_dict)

{'farm1': {'v1': 220}}


In [17]:
import paho.mqtt.client as mqtt
import time, json, datetime, sys, os

farm_data = {'farm1':{},
            'farm2':{},
            'farm3':{},
            'farm4':{},
            'farm5':{},
            }
#==================
#== MQTT Functions

# 當地端程式連線伺服器得到回應時，要做的動作
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    # 將訂閱主題寫在on_connet中
    # 如果我們失去連線或重新連線時 
    # 地端程式將會重新訂閱
    client.subscribe('KH20221202_IoT_Data_Science/#')
    

# 當接收到從伺服器發送的訊息時要進行的動作
def on_message(client, userdata, msg):
    # 轉換編碼utf-8才看得懂中文
    try:
        topic = msg.topic
        value = msg.payload.decode('utf-8')
        dtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print('(topic, value)=(%s, %s) @%s' %(topic, value, dtime))
        
        items = topic.split('/') #KH20221202_IoT_Data_Science/farm5/v3 -> [KH20221202_IoT_Data_Science, farm5, v3]
        farm_id = items[1]
        farm_sensor = items[2]
        farm_data[farm_id][farm_sensor] = value
    except Exception as e:
        print('read data exception: ', str(e))

def run_mqtt():
    try:
        # 連線設定
        # 初始化地端程式
        client = mqtt.Client()

        # 設定連線的動作
        client.on_connect = on_connect

        # 設定接收訊息的動作
        client.on_message = on_message

        # 設定登入帳號密碼
        #client.username_pw_set("try","xxxx")

        # 設定連線資訊(IP, Port, 連線時間)
        client.connect("broker.hivemq.com", 1883, 60)

        # 開始連線，執行設定的動作和處理重新連線問題
        # 也可以手動使用其他loop函式來進行連接
        #client.loop_forever()
        client.loop_start()    #start the loop

        time.sleep(30)

        client.disconnect() # disconnect gracefully
        client.loop_stop() # stops network loop
        
    except Exception as e:
        print('run mqtt error: ', str(e))

run_mqtt()


Connected with result code 0
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a1, 2.258) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm1/total_e, 5587.494) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v2, 234.660) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v3, 238.260) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a1, ) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a3, ) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm2/total_e, ) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm3/a1, 9.582) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm3/a2, 9.844) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm3/v1, 137.324) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm3/v2, 133.960) @2022-12-24 13:13:40
(topic, value)=(KH20221202_IoT_Data_Science/farm3/v3, 136.639) 

In [24]:
farm_data

{'farm1': {'a1': '2.282',
  'total_e': '5587.496',
  'v2': '234.245',
  'v3': '238.247',
  'a2': '2.330',
  'a3': '2.177',
  'v1': '236.291'},
 'farm2': {'a1': '24.033',
  'a3': '25.046',
  'total_e': '18707.470',
  'a2': '25.399',
  'v1': '231.012',
  'v2': '234.762',
  'v3': '235.971'},
 'farm3': {'a1': '9.641',
  'a2': '9.907',
  'v1': '137.670',
  'v2': '134.013',
  'v3': '136.509',
  'a3': '1.616',
  'total_e': '46493.040'},
 'farm4': {'a1': '0.109',
  'a2': '0.000',
  'v2': '124.255',
  'v1': '206.789',
  'total_e': '14614.280',
  'v3': '124.212',
  'a3': '0.325'},
 'farm5': {'a2': '0.000',
  'a3': '0.211',
  'total_e': '239.525',
  'a1': '0.033',
  'v1': '204.671',
  'v2': '78.735',
  'v3': '116.401'}}

In [25]:
farm_data['farm3']

{'a1': '9.641',
 'a2': '9.907',
 'v1': '137.670',
 'v2': '134.013',
 'v3': '136.509',
 'a3': '1.616',
 'total_e': '46493.040'}

In [20]:
# TODO: 請把電錶資訊以地圖的方式標示出來
#farm1: 23.6671309,120.1578337
#farm2: 23.6611758,120.161289
#farm3: 23.6355644,120.1474287
#farm4: 23.6373534,120.169344
#farm5: 23.6876348,120.2014575
meter_gps = {'farm1':[23.6671309,120.1578337],
            'farm2':[23.6611758,120.161289],
            'farm3':[23.6355644,120.1474287],
            'farm4':[23.6373534,120.169344],
            'farm5':[23.6876348,120.2014575],
            }



In [26]:
for key in meter_gps:
    print(key)

farm1
farm2
farm3
farm4
farm5


In [37]:
import folium
import geocoder

# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('雲林縣四湖鄉').latlng

m = folium.Map(location=location, zoom_start=12)

for key in meter_gps:
    try:
        # item[2]=緯度, item[1]=經度, item[0]=名稱
        name = key  #站名
        lat = meter_gps[key][0]  #緯度
        lng = meter_gps[key][1]  #經度
        total_e = farm_data[key]['total_e']
        v1 = farm_data[key]['v1']
        v2 = farm_data[key]['v2']
        v3 = farm_data[key]['v3']
        a1 = farm_data[key]['a1']
        a2 = farm_data[key]['a2']
        a3 = farm_data[key]['a3']
        info = '[%s] 用電%s度\r\n 電壓(%s,%s,%s) 電流(%s,%s,%s)' %(name, total_e, v1, v2, v3, a1, a2, a3)
        folium.Marker([ lat, lng],
                      tooltip=info,
                      #icon=folium.Icon(color='blue', prefix='fa', icon='fa-bicycle')
                      icon=folium.Icon(color='blue')
                     ).add_to(m)
        
    except Exception as e:
        print(e.args)    


In [38]:
m

In [40]:
import folium
import geocoder

# 使用 geocoder 取得特定住址的 GPS 座標
location = geocoder.osm('雲林縣四湖鄉').latlng

m = folium.Map(location=location, zoom_start=12)

for key in meter_gps:
    try:
        # item[2]=緯度, item[1]=經度, item[0]=名稱
        name = key  #站名
        lat = meter_gps[key][0]  #緯度
        lng = meter_gps[key][1]  #經度
        total_e = farm_data[key]['total_e']
        v1 = farm_data[key]['v1']
        v2 = farm_data[key]['v2']
        v3 = farm_data[key]['v3']
        a1 = farm_data[key]['a1']
        a2 = farm_data[key]['a2']
        a3 = farm_data[key]['a3']
        info = '[%s] 用電%s度\r\n 電壓(%s,%s,%s) 電流(%s,%s,%s)' %(name, total_e, v1, v2, v3, a1, a2, a3)
        if float(v1)>238 or float(v2)>238 or float(v3)>238:
            folium.Marker([ lat, lng],
                          tooltip=info,
                          #icon=folium.Icon(color='blue', prefix='fa', icon='fa-bicycle')
                          icon=folium.Icon(color='red')
                         ).add_to(m)
        else:
            folium.Marker([ lat, lng],
                          tooltip=info,
                          #icon=folium.Icon(color='blue', prefix='fa', icon='fa-bicycle')
                          icon=folium.Icon(color='blue')
                         ).add_to(m)
        
    except Exception as e:
        print(e.args)    


In [41]:
m

----

## Web API

適合非即時性的應用

----


----

### 運用 ThingSpeak收集資料

- https://thingspeak.com/
- 註冊登入後，於`https://thingspeak.com/channels`可以看到你的管理畫面
- [文件](https://indico.cern.ch/event/654636/contributions/2834242/attachments/1636045/2610243/MATLAB_IoT_public.pdf)
----

In [47]:
# continue push data

my_h = [60, 70, 60, 80, 75, 85]
my_t = [25, 27, 28, 30, 28, 25]

In [48]:
len(my_t)

6

In [50]:
import requests
import time

for i in range(len(my_t)):
    
    print('---- %s ----' %(i))
    print('H=%s %%, T=%s degree' %(my_h[i], my_t[i]))
    
    # push data
    # field1: T
    # field2: H
    url = 'https://api.thingspeak.com/update'
    api_key = '1FRPKJGL975ZWPQP'
    field1 = my_t[i]
    field2 = my_h[i]
    data = {'api_key': api_key, 'field1':field1, 'field2':field2}
    r = requests.get(url, params=data)
    print(r)
    
    time.sleep(15)
    

---- 0 ----
H=60 %, T=25 degree
<Response [200]>
---- 1 ----
H=70 %, T=27 degree
<Response [200]>
---- 2 ----
H=60 %, T=28 degree
<Response [200]>
---- 3 ----
H=80 %, T=30 degree
<Response [200]>
---- 4 ----
H=75 %, T=28 degree
<Response [200]>
---- 5 ----
H=85 %, T=25 degree
<Response [200]>


In [51]:
import paho.mqtt.client as mqtt
import time, json, datetime, sys, os

farm_data = {'farm1':{},
            'farm2':{},
            'farm3':{},
            'farm4':{},
            'farm5':{},
            }
field1 = None
field2 = None
field3 = None
#==================
#== MQTT Functions

# 當地端程式連線伺服器得到回應時，要做的動作
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    # 將訂閱主題寫在on_connet中
    # 如果我們失去連線或重新連線時 
    # 地端程式將會重新訂閱
    client.subscribe('KH20221202_IoT_Data_Science/#')
    

# 當接收到從伺服器發送的訊息時要進行的動作
def on_message(client, userdata, msg):
    # 轉換編碼utf-8才看得懂中文
    try:
        topic = msg.topic
        value = msg.payload.decode('utf-8')
        dtime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        print('(topic, value)=(%s, %s) @%s' %(topic, value, dtime))
        
        items = topic.split('/') #KH20221202_IoT_Data_Science/farm5/v3 -> [KH20221202_IoT_Data_Science, farm5, v3]
        farm_id = items[1]
        farm_sensor = items[2]
        farm_data[farm_id][farm_sensor] = value
        
        # push data to thingSpeak
        # 5TG299PV5MM0P3H4
        if farm_id == 'farm2':
            url = 'https://api.thingspeak.com/update'
            api_key = '5TG299PV5MM0P3H4'
            if farm_sensor == 'v1':
                field1 = value
            if farm_sensor == 'v2':
                field2 = value
            if farm_sensor == 'v3':
                field3 = value
            if (field1 != None) and (field2 != None) and (field3 != None):
                data = {'api_key': api_key, 'field1':field1, 'field2':field2, 'field2':field2}
                r = requests.get(url, params=data)
                print(r)
        
    except Exception as e:
        print('read data exception: ', str(e))

def run_mqtt():
    try:
        # 連線設定
        # 初始化地端程式
        client = mqtt.Client()

        # 設定連線的動作
        client.on_connect = on_connect

        # 設定接收訊息的動作
        client.on_message = on_message

        # 設定登入帳號密碼
        #client.username_pw_set("try","xxxx")

        # 設定連線資訊(IP, Port, 連線時間)
        client.connect("broker.hivemq.com", 1883, 60)

        # 開始連線，執行設定的動作和處理重新連線問題
        # 也可以手動使用其他loop函式來進行連接
        #client.loop_forever()
        client.loop_start()    #start the loop

        time.sleep(50)

        client.disconnect() # disconnect gracefully
        client.loop_stop() # stops network loop
        
    except Exception as e:
        print('run mqtt error: ', str(e))

run_mqtt()


Connected with result code 0
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a1, 0.406) @2022-12-24 14:37:01
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a2, 0.397) @2022-12-24 14:37:01
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a3, 0.037) @2022-12-24 14:37:01
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v1, 237.842) @2022-12-24 14:37:01
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v2, 237.394) @2022-12-24 14:37:01
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v3, 239.051) @2022-12-24 14:37:01
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a2, ) @2022-12-24 14:37:01
read data exception:  local variable 'field1' referenced before assignment
(topic, value)=(KH20221202_IoT_Data_Science/farm2/total_e, ) @2022-12-24 14:37:01
read data exception:  local variable 'field1' referenced before assignment
(topic, value)=(KH20221202_IoT_Data_Science/farm2/v1, ) @2022-12-24 14:37:01
read data exception:  local variable 'field2' referenced before assignment
(topic, valu

(topic, value)=(KH20221202_IoT_Data_Science/farm5/v2, 78.715) @2022-12-24 14:37:22
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a1, 0.407) @2022-12-24 14:37:32
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a2, 0.399) @2022-12-24 14:37:32
(topic, value)=(KH20221202_IoT_Data_Science/farm1/a3, 0.038) @2022-12-24 14:37:32
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v1, 237.594) @2022-12-24 14:37:32
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v2, 237.283) @2022-12-24 14:37:32
(topic, value)=(KH20221202_IoT_Data_Science/farm1/v3, 238.659) @2022-12-24 14:37:32
(topic, value)=(KH20221202_IoT_Data_Science/farm2/a2, ) @2022-12-24 14:37:32
read data exception:  local variable 'field1' referenced before assignment
(topic, value)=(KH20221202_IoT_Data_Science/farm2/total_e, ) @2022-12-24 14:37:32
read data exception:  local variable 'field1' referenced before assignment
(topic, value)=(KH20221202_IoT_Data_Science/farm2/v1, ) @2022-12-24 14:37:32
read data exception:  local varia